In [3]:
# Manejo de arreglos y operaciones matematicas
import numpy as np
# Numeros aleatorios
import random

In [4]:
def pg_best(X, y, y_hat, X_ft, y_ft, y_hat_ft):
    """
    Esta funcion actualiza el personal best de cada particula y el global best
    - input: 
        - X: poblacion
        - y: personal best actual de cada particula
        - y_hat: global best
        - X_ft: fitness de la poblacion
        - y_ft: fitness del personal best
        - y_hat_ft: fitness del mejor global
    - output: personal best y global best actualizados
    """
    # Obtener el indice de las particulas donde su nueva posicion es mejor que su personal best
    idx_mejoresp = np.where(X_ft < y_ft)
    
    # Actualizar personal best
    y[idx_mejoresp] = X[idx_mejoresp]
    
    # Si se encuentra un valor mejor en la poblacion actualizada, se cambia el gbest
    if np.min(X_ft) < y_hat_ft:
        y_hat = X[[np.argmin(X_ft)]]
    
    return y, y_hat

def actualizar_vel(v, X, y, y_hat, w, c1, c2):
    """
    Esta funcion actualiza la velocidad de las particulas
    - input:
        - v: velocidad actual de las particulas
        - X: posicion actual de las particulas
        - y: personal best de cada particula
        - y_hat: global best
        - c1, c2: constantes para peso de la componente personal y social
    - output: velocidad actualizada de cada particula
    """
    # Calcular nueva velocidad
    v = w * v + (c1 * np.random.uniform(0, 1, (len(X), 1)) * (y - X)) + (c2 * np.random.uniform(0, 1, (len(X), 1)) * (y_hat - X))
    
    return v

def PSO_gbest(d, funcion, rango, n, w, c1, c2, max_gen, tol):    
    # Si solo hay un rango se considera el mismo rango para todas las variables
    if len(rango) == 1:
        rango *= d
    # Convertir rango a np.array
    rango = np.array(rango)

    # Definimos un mejor fitness grande para iniciar el algoritmo
    mejor_fitness= np.infty
    gen = 0

    # Crear un vetor de soluciones para cada dimension con su rango dado
    X = np.random.uniform(low=rango[:, 0], high=rango[:,1], size=((n, d)))

    # El personal best se inicializa con la poblacion ya que no ha habido actualizacion
    y = X.copy() 

    # Inicializar el global best con el minimo de la poblacion inicial
    y_hat = X[[np.argmin(funcion(X))]]

    # Incializar aleatoriamente la velocidad de las particulas
    v = np.random.uniform(low=rango[:, 0], high=rango[:,1], size=((n, d)))

    # Seguir con el algoritmo mientra sean menos de 200 generaciones o el fitness sea menor a 0.001
    while((mejor_fitness > tol) and (gen < max_gen)):
        # Calcular el fitness del personal best
        y_ft = funcion(y)

        # Fitness del mejor global
        y_hat_ft = funcion(y_hat)

        # Fitness de la poblacion
        X_ft = funcion(X)

        # Actualizar el personal y global best
        y, y_hat = pg_best(X, y, y_hat, X_ft, y_ft, y_hat_ft)

        mejor_fitness = funcion(y_hat)[0]

        # Actualizar velocidad
        v = actualizar_vel(v, X, y, y_hat, w, c1, c2)

        # Actualizar la posicion de las particulas
        X += v

        # Clipear para evitar que los valores se salgan del rango
        X = np.clip(X, a_min=rango[:, 0], a_max=rango[:, 1])

        # Aumentar la generacion
        gen += 1

    # Una vez concluido el algoritmo se muestran los resultados
    print(f"El óptimo está en {np.around(y_hat[0], 3)} con un fitness de {mejor_fitness:0.4f}")
    print(f"Obtenido en la generación {gen}")
    return y_hat[0]

def crear_vecindarios(X, k):
    """
    Esta funcion crea los vecindarios de k vecinos para las n particulas
    - input:
        - X: poblacion
        - k: numero de vecinos de cada particula
    """
    # Se puede optimizar?
    # Checar si el numero de vecinos es valido
    if (k % 2 != 0) or k > len(X):
        print("Ingrese un número de vecinos valido (par y menor o igual al numero de particulas)")
        return
    else:
        vecindarios = []
        # Crear vecindarios
        for i in range(len(X)):
            vecindarios.append(np.arange(i-k/2, i+k/2+1))  # Los vecinos son los k/2 anteriores y los k/siguientes
        vecinos = np.array(vecindarios).astype(int)
        # Poner todos los indices dentro del rango valido para los indices
        vecinos[vecinos > len(X) -1] -= len(X)
        return vecinos
    
def obtener_mejorn(X, neigh, funcion):
    """
    Esta funcione obtiene el mejor de cada vecindario
    input: 
        - X: poblacion
        - neigh: vecindarios de cada particula
        - funcion: funcion objetivo
    output: lista con los mejores de cada vecindario
    """
    # Lista para colocar los mejores de cada vecindario
    y_hats = np.zeros_like(X)
    for i in range(len(neigh)):
        N = X[neigh[i]]  # Miembros del vecindario
        y_hat = N[np.argmin(funcion(N))]  # Obtener el mejor del vecindario i
        y_hats[i] = y_hat.reshape(-1, X.shape[1])  # Actualizar el mejor del vecindario
    return y_hats

def pl_best(X, y, y_hat, X_ft, y_ft, neigh, funcion):
    """
    Esta funcion actualiza el personal best de cada particula y el mejor de cada vecindario
    - input: 
        - X: poblacion
        - y: personal best actual de cada particula
        - y_hat: global best
        - X_ft: fitness de la poblacion
        - y_ft: fitness del personal best
        - neigh: vecindarios
        - funcion: funcion objetivo
    - output: personal best y global best actualizados
    """
    # Obtener el indice de las particulas donde su nueva posicion es mejor que su personal best
    idx_mejoresp = np.where(X_ft < y_ft)
    
    # Actualizar personal best
    y[idx_mejoresp] = X[idx_mejoresp]
    
    # Obtener el fitness de los vecindarios actualizados
    y_hat_temp = obtener_mejorn(X, neigh, funcion)
    
    # Obtener el indice de las particulas del vecindario donde su nueva posicion es mejor que su mejor del neigh
    idx_mejoresn = np.where(funcion(y_hat_temp) < funcion(y_hat))
    
    # Actualizar los mejores de los vecindarios
    y_hat[idx_mejoresn] = y_hat_temp[idx_mejoresn]
    
    return y, y_hat

def PSO_lbest(d, funcion, rango, n, k, w, c1, c2, max_gen, tol):
    # Si solo hay un rango se considera el mismo rango para todas las variables
    if len(rango) == 1:
        rango *= d
    # Convertir rango a np.array
    rango = np.array(rango)

    # Definimos un mejor fitness grande para iniciar el algoritmo
    mejor_fitness= np.infty
    gen = 0

    # Crear un vetor de soluciones para cada dimension con su rango dado
    X = np.random.uniform(low=rango[:, 0], high=rango[:,1], size=((n, d)))

    # El personal best se inicializa con la poblacion ya que no ha habido actualizacion
    y = X.copy() 

    # Crear vecindarios
    neigh = crear_vecindarios(X, k)

    # Inicializar el global best con el minimo de cada vecindario
    y_hat = obtener_mejorn(X, neigh, funcion)

    # Incializar aleatoriamente la velocidad de las particulas
    v = np.random.uniform(low=rango[:, 0], high=rango[:,1], size=((n, d)))

    # Seguir con el algoritmo mientra sean menos de 200 generaciones o el fitness sea menor a 0.001
    while((mejor_fitness > tol) and (gen < max_gen)):
        # Calcular el fitness del personal best
        y_ft = funcion(y)

        # Fitness del mejor global
        y_hat_ft = funcion(y_hat)

        # Fitness de la poblacion
        X_ft = funcion(X)

        # Actualizar el personal y global best
        y, y_hat = pl_best(X, y, y_hat, X_ft, y_ft, neigh, funcion)

        # Actualizar el mejor fitness actual
        mejor_fitness_idx = np.argmin(funcion(y_hat))  # idx del mejor fitness
        mejor_fitness = funcion(y_hat)[mejor_fitness_idx]  # valor del mejor fitness
        mejor_part = y_hat[mejor_fitness_idx]  # Mejor particula

        # Actualizar velocidad
        v = actualizar_vel(v, X, y, y_hat, w, c1, c2)

        # Actualizar la posicion de las particulas
        X += v

        # Clipear para evitar que los valores se salgan del rango
        X = np.clip(X, a_min=rango[:, 0], a_max=rango[:, 1])

        # Aumentar la generacion
        gen += 1

    # Una vez concluido el algoritmo se muestran los resultados
    print(f"El óptimo está en {np.around(mejor_part, 3)} con un fitness de {mejor_fitness:0.4f}")
    print(f"Obtenido en la generación {gen}")
    return mejor_part